# training

## setup

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/tiger_detection/

/content/drive/MyDrive/tiger_detection


In [ ]:
!pip3 install keras kimm -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 13.1 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1


In [ ]:
import numpy as np
import keras
from keras import layers
import kimm
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

## prepare train and test

In [ ]:
# Define paths to train and test datasets
train_dir = "images/train"
test_dir = "images/test"
test2_dir = "images/test2"

# Create training dataset
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    label_mode='int',
    shuffle=True
)

# Create testing dataset
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    label_mode='int',
    shuffle=False
)
test2_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test2_dir,
    label_mode='int',
    shuffle=False
)

# # Optional: Normalize the pixel values (between 0 and 1)
# normalization_layer = tf.keras.layers.Rescaling(1./255)

# train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
# test_dataset = test_dataset.map(lambda x, y: (normalization_layer(x), y))

# we need to unbatch because there's somehow an unwanted additional dimension
train_ds = train_ds.unbatch()
test_ds = test_ds.unbatch()
test2_ds = test2_ds.unbatch()

print(f"Number of training samples: {train_ds.cardinality()}")
#print(f"Number of validation samples: {validation_ds.cardinality()}")
print(f"Number of test samples: {test_ds.cardinality()}")

Found 6000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.
Found 320 files belonging to 1 classes.
Number of training samples: -2
Number of test samples: -2


In [ ]:
print(train_ds.element_spec)

(TensorSpec(shape=(256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int32, name=None))


In [ ]:
print(test_ds.element_spec)

(TensorSpec(shape=(256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int32, name=None))


In [ ]:
# Setup dataset with tf.data
resize_fn = keras.layers.Resizing(150, 150)

train_ds = train_ds.map(lambda x, y: (resize_fn(x), y))
#validation_ds = validation_ds.map(lambda x, y: (resize_fn(x), y))
test_ds = test_ds.map(lambda x, y: (resize_fn(x), y))
test2_ds = test2_ds.map(lambda x, y: (resize_fn(x), y))
# augmentation_layers = [
#     layers.RandomFlip("horizontal"),
#     layers.RandomRotation(0.1),
# ]


# def data_augmentation(x):
#     for layer in augmentation_layers:
#         x = layer(x)
#     return x


# train_ds = train_ds.map(lambda x, y: (data_augmentation(x), y))

batch_size = 64
train_ds = train_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE).cache()
#validation_ds = validation_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE).cache()
test_ds = test_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE).cache()
test2_ds = test2_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE).cache()

In [ ]:
# kimm will load weights pre-trained on ImageNet by default in most cases
# You can try different model such as ConvNeXtAtto, EfficientNetLiteB0, GhostNet100, RegNetY002, etc.
base_model = kimm.models.EfficientNetLiteB0(
    input_shape=(150, 150, 3),
    include_preprocessing=True,
    include_top=False,
)

# Freeze the base_model
base_model.trainable = False

# Create new model on top
inputs = keras.Input(shape=(150, 150, 3))
x = inputs

# The base model contains batchnorm layers. We want to keep them in inference mode
# when we unfreeze the base model for fine-tuning, so we make sure that the
# base_model is running in inference mode here.
x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

model.summary(show_trainable=True)

19141320/19141320 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ Layer (type)                        ┃ Output Shape                 ┃       Param # ┃ Traina… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ input_layer_1 (InputLayer)          │ (None, 150, 150, 3)          │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ EfficientNetLiteB0                  │ (None, 5, 5, 1280)           │     3,413,024 │    N    │
│ (EfficientNetLiteB0)                │                              │               │         │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ global_average_pooling2d            │ (None, 1280)                 │             0 │    -    │
│ (GlobalAveragePooling2D)            │                              │               │         │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ dropout (Dropout)                   │ (None, 1280)                 │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ dense (Dense)                       │ (None, 1)                    │         1,281 │    Y    │
└─────────────────────────────────────┴──────────────────────────────┴───────────────┴─────────┘

 Total params: 3,414,305 (13.02 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 3,413,024 (13.02 MB)

## training

In [ ]:
# You should achieve good results in 2 epochs
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)

epochs = 2
print("Fitting the top layer of the model")
model.fit(train_ds, epochs=epochs)#, validation_data=validation_ds)

result = model.evaluate(test_ds)
print(f"Test accuracy: {result[1] * 100:3.2f}%")

Fitting the top layer of the model
Epoch 1/2
94/94 ━━━━━━━━━━━━━━━━━━━━ 1217s 12s/step - binary_accuracy: 0.8107 - loss: 0.3437
Epoch 2/2
 6/94 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - binary_accuracy: 0.9580 - loss: 0.1268

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


94/94 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - binary_accuracy: 0.9539 - loss: 0.1185
32/32 ━━━━━━━━━━━━━━━━━━━━ 756s 24s/step - binary_accuracy: 0.9790 - loss: 0.0751
Test accuracy: 97.50%


In [ ]:
result2 = model.evaluate(test2_ds)
print(f"Test2 accuracy: {result2[1] * 100:3.2f}%")

5/5 ━━━━━━━━━━━━━━━━━━━━ 114s 21s/step - binary_accuracy: 0.9036 - loss: 0.2682
Test2 accuracy: 89.69%
